<a href="https://colab.research.google.com/github/incheonQ/AIFFEL_quest_cr/blob/main/%EC%9C%A0%ED%86%B5%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%AA%A8%EB%8D%B8%EB%A7%81Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 데이터준비
- 데이터분할
- 모델 훈련
- 하이퍼파라미터 튜닝
- 성능 평가


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 로드
cli_data = pd.read_excel('/content/drive/MyDrive/df_merge2.xlsx')
#cpi_data = pd.read_excel('/content/drive/MyDrive/cpi.xlsx')
#eco_data = pd.read_excel('/content/drive/MyDrive/경제지표.xlsx')

print(cli_data.head())
#print(cpi_data.head())
#print(eco_data.head())

         판매일  판매수량  소분류  브랜드  파레토  라면_검색어트렌드  통조림_검색어트렌드  즉석상온_검색어트렌드  \
0 2021-01-04     3   라면   농심    1   43.84615     9.04761      6.63285   
1 2021-01-04     1   라면   농심    1   43.84615     9.04761      6.63285   
2 2021-01-04     1   라면   농심    1   43.84615     9.04761      6.63285   
3 2021-01-04     1  통조림   동원    1   43.84615     9.04761      6.63285   
4 2021-01-04     1   라면  오뚜기    1   43.84615     9.04761      6.63285   

   평균기온(℃)  최고기온 평균(℃)  최저기온 평균(℃)  강수량(mm)  눈 일수(일)  폭염일수(일)  한파일수(일)  \
0     -2.4         2.2        -6.8     18.9        9        0        5   
1     -2.4         2.2        -6.8     18.9        9        0        5   
2     -2.4         2.2        -6.8     18.9        9        0        5   
3     -2.4         2.2        -6.8     18.9        9        0        5   
4     -2.4         2.2        -6.8     18.9        9        0        5   

       연월    상품 바코드(대한상의)                  상품명         날짜  
0  2021_1   8801043045674         농심]안성탕면컵<6入> 2021-01-0

In [ ]:
cli_data['판매일'] = pd.to_datetime(cli_data['판매일'])
#cpi_data['날짜'] = pd.to_datetime(cpi_data['날짜'])
#eco_data['날짜'] = pd.to_datetime(eco_data['날짜'])
#data = cli_data.merge(eco_data, on='날짜')

cli_data['연도'] = cli_data['날짜'].dt.year
cli_data['월'] = cli_data['날짜'].dt.month
cli_data['일'] = cli_data['날짜'].dt.day
cli_data['요일'] = cli_data['날짜'].dt.weekday

data= cli_data

In [ ]:
data.columns

Index(['판매일', '판매수량', '소분류', '브랜드', '파레토', '라면_검색어트렌드', '통조림_검색어트렌드',
       '즉석상온_검색어트렌드', '평균기온(℃)', '최고기온 평균(℃)', '최저기온 평균(℃)', '강수량(mm)',
       '눈 일수(일)', '폭염일수(일)', '한파일수(일)', '연월', '상품 바코드(대한상의)', '상품명', '날짜',
       '연도', '월', '일', '요일'],
      dtype='object')

In [ ]:
data.dtypes

,0
판매일,datetime64[ns]
판매수량,int64
소분류,object
브랜드,object
파레토,int64
라면_검색어트렌드,float64
통조림_검색어트렌드,float64
즉석상온_검색어트렌드,float64
평균기온(℃),float64
최고기온 평균(℃),float64


In [ ]:
data.drop(['연월','날짜','판매일','상품명','브랜드'], axis=1, inplace=True)

In [ ]:
data = pd.get_dummies(data, columns=['소분류'], drop_first=True)
data.head(5)

,판매수량,파레토,라면_검색어트렌드,통조림_검색어트렌드,즉석상온_검색어트렌드,평균기온(℃),최고기온 평균(℃),최저기온 평균(℃),강수량(mm),눈 일수(일),폭염일수(일),한파일수(일),상품 바코드(대한상의),연도,월,일,요일,소분류_상온즉석,소분류_통조림
0,3,1,43.84615,9.04761,6.63285,-2.4,2.2,-6.8,18.9,9,0,5,8801043045674,2021,1,1,4,False,False
1,1,1,43.84615,9.04761,6.63285,-2.4,2.2,-6.8,18.9,9,0,5,8801043015028,2021,1,1,4,False,False
2,1,1,43.84615,9.04761,6.63285,-2.4,2.2,-6.8,18.9,9,0,5,8801043014847,2021,1,1,4,False,False
3,1,1,43.84615,9.04761,6.63285,-2.4,2.2,-6.8,18.9,9,0,5,18801047141140,2021,1,1,4,False,True
4,1,1,43.84615,9.04761,6.63285,-2.4,2.2,-6.8,18.9,9,0,5,18801045522286,2021,1,1,4,False,False


In [ ]:

X = data.drop('판매수량', axis=1)
y = data['판매수량']

train_data = data[data['연도'] < 2023] # 2021~2022를 훈련데이터로
test_data = data[data['연도'] >= 2023] # 2023 을 테스트 데이터로 활용

X_train = train_data.drop('판매수량', axis=1)
y_train = train_data['판매수량']
X_test = test_data.drop('판매수량', axis=1)
y_test = test_data['판매수량']

In [ ]:
# XGBoost 모델
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# LightGBM 모델
lgb_model = LGBMRegressor()
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021441 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1131
[LightGBM] [Info] Number of data points in the train set: 134698, number of used features: 17
[LightGBM] [Info] Start training from score 2.708080


In [ ]:
print("XGBoost MAE:", mean_absolute_error(y_test, y_pred_xgb))
print("XGBoost MSE:", mean_squared_error(y_test, y_pred_xgb))

print("LightGBM MAE:", mean_absolute_error(y_test, y_pred_lgb))
print("LightGBM MSE:", mean_squared_error(y_test, y_pred_lgb))

XGBoost MAE: 1.768398248583615
XGBoost MSE: 33.09182985918176
LightGBM MAE: 1.680543074138366
LightGBM MSE: 29.39893404672232
